# Performance

*Julia* avec la compilation *Just-In-Time* est un langage naturellement performant. Il n'est pas allergique aux boucles comme le sont les langages Python et R. Les opérations vectorisées fonctionnent également très bien à condition d'être attentifs aux allocations mémoire et aux vues explicites.

## Allocations

In [2]:
using Random, LinearAlgebra, BenchmarkTools

function test(A, B, C)
    C = C - A * B
    return C
end

A = rand(1024, 256); B = rand(256, 1024); C = rand(1024, 1024)

@btime test($A, $B, $C);

  5.443 ms (4 allocations: 16.00 MiB)


Dans l'appel de la macro `@benchmark` on interpole les arguments avec le signe `$` pour être sur que les fonctions
    `rand` aient déjà été evaluées avant l'appel de la fonction `test`. La matrice `C` est modifiée dans la fonction suivante donc par convention on ajoute un `!` au nom de la fonction. Par convention également, l'argument modifié se placera en premier. Comme dans la fonction `push!` par exemple.

In [7]:
function test!(C, A, B)
    C .= C .- A * B
    return C
end

@btime test!( $C, $A, $B);

  4.898 ms (2 allocations: 8.00 MiB)


En effectuant une opération "en place", on supprime une allocation mais celle pour effectuer l'opération `A * B` est toujours nécessaire. On peut supprimer cette allocation en utilisant la bibliothèque `BLAS`, cependant le code perd en lisibilité ce qu'il a gagné en performance.

In [5]:
function test_opt!(C, A, B)
    BLAS.gemm!('N','N', -1., A, B, 1., C)
    return C
end

@btime test_opt!($C, $A, $B);

  3.003 ms (0 allocations: 0 bytes)


## Alignement de la mémoire

Les opérations le long des premiers indices seront plus rapides.

In [9]:
using FFTW

T = randn(1024, 1024)

@btime fft($T, 1);

  8.291 ms (29 allocations: 32.00 MiB)


In [10]:
@btime fft($T, 2);

  33.095 ms (29 allocations: 32.00 MiB)


Voici un autre exemple ou on calcule la dérivée de la quantité $f$ suivant la coordonnée $y$ en passant par l'espace de Fourier

In [12]:
using FFTW

xmin, xmax, nx = 0, 4π, 1024
ymin, ymax, ny = 0, 4π, 1024
x = LinRange(xmin, xmax, nx+1)[1:end-1]
y = LinRange(ymin, ymax, ny+1)[1:end-1]
ky  = 2π ./ (ymax-ymin) .* [0:ny÷2-1;ny÷2-ny:-1]
exky = exp.( 1im .* ky' .* x)
function df_dy( f, exky )
    ifft(exky .* fft(f, 2), 2)
end
f = sin.(x) .* cos.(y') # f is a 2d array created by broadcasting
@btime df_dy($f, $exky);

  71.749 ms (59 allocations: 64.00 MiB)


En utilisant les "plans" de FFTW qui permettent de pré-allouer la mémoire nécessaire et le calcul "en place". On peut améliorer les performances. On réutilise le même tableau pour la valeur de $f$ et sa transformée de Fourier. On prend soin également de respecter l'alignement de la mémoire en transposant le tableau contenant $f$ pour calculer la FFT. On utilise plus de mémoire, on fait plus de calcul en ajoutant les transpositions mais finalement le calcul va 3 fois plus vite car on évite les allocations et on limite les accès mémoire.

In [13]:
f  = zeros(ComplexF64, (nx,ny))
fᵗ = zeros(ComplexF64, reverse(size(f)))
f̂ᵗ = zeros(ComplexF64, reverse(size(f)))
f .= sin.(x) .* cos.(y')
fft_plan = plan_fft(fᵗ, 1, flags=FFTW.PATIENT)
function df_dy!( f, fᵗ, f̂ᵗ, exky )
    transpose!(fᵗ,f)
    mul!(f̂ᵗ,  fft_plan, fᵗ)
    f̂ᵗ .= f̂ᵗ .* exky
    ldiv!(fᵗ, fft_plan, f̂ᵗ)
    transpose!(f, fᵗ)
end
@btime df_dy!($f, $fᵗ, $f̂ᵗ, $exky );

  18.432 ms (2 allocations: 96 bytes)


## Vues explicites

In [14]:
@btime sum(T[:,1]) # Somme de la première colonne

  1.163 μs (3 allocations: 8.16 KiB)


-37.45424678271367

In [15]:
@btime sum(view(T,:,1))  

  229.237 ns (3 allocations: 80 bytes)


-37.45424678271367

## Eviter les calculs dans l'environnement global.

In [18]:
v = rand(1000)

function somme()
    acc = 0
    for i in eachindex(v) 
        acc += v[i]
    end
    acc
end

@btime somme()


  86.435 μs (3978 allocations: 77.77 KiB)


512.5043541109563

In [23]:
@code_lowered somme()

CodeInfo(
1 ─       acc = 0
│   %2  = Main.eachindex(Main.v)
│         @_2 = Base.iterate(%2)
│   %4  = @_2 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_2
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│   %10 = acc
│   %11 = Base.getindex(Main.v, i)
│         acc = %10 + %11
│         @_2 = Base.iterate(%2, %9)
│   %14 = @_2 === nothing
│   %15 = Base.not_int(%14)
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return acc
)

Il faut écrire des fonctions

In [20]:
function somme( x )
    acc = 0
    for i in eachindex(x) 
        acc += x[i]
    end
    acc
end

@btime somme( v )
    

  2.300 μs (1 allocation: 16 bytes)


512.5043541109563

In [22]:
@code_lowered somme(v)

CodeInfo(
1 ─       acc = 0
│   %2  = Main.eachindex(x)
│         @_3 = Base.iterate(%2)
│   %4  = @_3 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_3
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│   %10 = acc
│   %11 = Base.getindex(x, i)
│         acc = %10 + %11
│         @_3 = Base.iterate(%2, %9)
│   %14 = @_3 === nothing
│   %15 = Base.not_int(%14)
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return acc
)

Pour comprendre pourquoi l'utilisation de variable global influence les performances, prenons un exemple simple d'une fonction additionnant deux nombres:

In [24]:
variable = 10

function addition_variable_globale(x)
    x + variable
end

@btime addition_variable_globale(10)

  19.534 ns (0 allocations: 0 bytes)


20

Comparons la performance avec cette fonction qui retourne la somme de ses deux arguments

In [26]:
function addition_deux_arguments(x, y)
    x + y
end

@btime addition_deux_arguments(10, 10)

  0.041 ns (0 allocations: 0 bytes)


20

On remarque que la deuxième fonction est 300 fois plus rapide que la première. Pour comprendre pourquoi elle est plus rapide, on peut regarder le code généré avant la compilation. On s'appercoit que le code est relativement simple avec une utilisation unique de l'instruction `add`.

In [27]:
@code_llvm addition_deux_arguments(10, 10)

;  @ In[26]:1 within `addition_deux_arguments`
define i64 @julia_addition_deux_arguments_2713(i64 signext %0, i64 signext %1) #0 {
top:
;  @ In[26]:2 within `addition_deux_arguments`
; ┌ @ int.jl:87 within `+`
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


Si on regarde le code généré utilisant la variable globale, on comprend rapidement pourquoi c'est plus long. Pourquoi le code est-il si compliqué ? Ici le langage ne connait pas le type de `variable`, il doit donc prendre en compte le fait que ce type puisse être modifié à tout moment. Comme tous les cas sont envisagés, cela provoque un surcoût important.

In [28]:
@code_llvm addition_variable_globale(10)

;  @ In[24]:3 within `addition_variable_globale`
define nonnull {}* @julia_addition_variable_globale_2715(i64 signext %0) #0 {
top:
  %1 = alloca [2 x {}*], align 8
  %gcframe2 = alloca [4 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %1, i64 0, i64 0
  %2 = bitcast [4 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i32(i8* nonnull align 16 dereferenceable(32) %2, i8 0, i32 32, i1 false)
  %3 = call {}*** inttoptr (i64 140703307841493 to {}*** (i64)*)(i64 260) #3
;  @ In[24]:4 within `addition_variable_globale`
  %4 = bitcast [4 x {}*]* %gcframe2 to i64*
  store i64 8, i64* %4, align 16
  %5 = load {}**, {}*** %3, align 8
  %6 = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 1
  %7 = bitcast {}** %6 to {}***
  store {}** %5, {}*** %7, align 8
  %8 = bitcast {}*** %3 to {}***
  store {}** %gcframe2.sub, {}*** %8, align 8
  %9 = load atomic {}*, {}**

Il est donc possible d'améliorer la performance en fixant la valeur de la variable globale avec l'instruction `const`.

In [32]:
const constante = 10

function addition_variable_constante(x)
    x + constante
end

@btime addition_variable_constante(10)

  0.049 ns (0 allocations: 0 bytes)


20

On peut également fixer le type de cette variable. C'est mieux mais cela reste éloigné, en terme de performance, du résultat précedent.

In [30]:
function addition_variable_typee(x)
    x + variable::Int
end

@btime addition_variable_typee(10)

  1.861 ns (0 allocations: 0 bytes)


20

Pour régler notre problème de performance avec une variable globale, il faut la passer en argument dans la fonction.

In [ ]:
function addition_variable_globale_en_argument(x, v)
    x + v
end

In [ ]:
@btime addition_variable_globale_en_argument(10, $variable)

## Instabilité de type

Une fonction est de type stable lorsque vous pouvez déduire ce que doit être la sortie de la fonction. L'exemple ci-dessous rendra les choses plus claires. En règle générale, les fonctions de type stable sont plus rapides.


In [33]:
function carre_plus_un(v::T) where T <:Number
    g = v * v
    return g+1
end

carre_plus_un (generic function with 1 method)

In [34]:
v = rand()

0.79927465273255

In [35]:
@code_warntype carre_plus_un(v)

MethodInstance for carre_plus_un(::Float64)
  from carre_plus_un(v::T) where T<:Number in Main at In[33]:1
Static Parameters
  T = Float64
Arguments
  #self#::Core.Const(carre_plus_un)
  v::Float64
Locals
  g::Float64
Body::Float64
1 ─      (g = v * v)
│   %2 = (g + 1)::Float64
└──      return %2



In [36]:
w = 5

5

In [37]:
@code_warntype carre_plus_un(w)

MethodInstance for carre_plus_un(::Int64)
  from carre_plus_un(v::T) where T<:Number in Main at In[33]:1
Static Parameters
  T = Int64
Arguments
  #self#::Core.Const(carre_plus_un)
  v::Int64
Locals
  g::Int64
Body::Int64
1 ─      (g = v * v)
│   %2 = (g + 1)::Int64
└──      return %2



Sur les deux exemples précedents on peut déduire le type de sortie de la fonction.
```
function carre_plus_un(v::T) where T <:Number
    g = v*v         # Type(T * T) ==> T
    return g+1      # Type(T + Int)) ==> "max" (T,Int)
end

```
Le type de la valeur de retour peut être différent: `Float64` ou `Int64`. Mais la fonction est toujours stable.

Créons maintenant un nouveau type:

In [38]:
mutable struct Cube
    length
    width
    height
end

In [40]:
volume(c::Cube) = c.length*c.width*c.height

volume (generic function with 1 method)

In [41]:
mutable struct Cube_typed
    length::Float64
    width::Float64
    height::Float64
end
volume(c::Cube_typed) = c.length*c.width*c.height

volume (generic function with 2 methods)

In [42]:
mutable struct Cube_parametric_typed{T <: Real}
    length :: T
    width :: T
    height :: T
end
volume(c::Cube_parametric_typed) = c.length*c.width*c.height

volume (generic function with 3 methods)

In [43]:
c1 = Cube(1.1,1.2,1.3)
c2 = Cube_typed(1.1,1.2,1.3)
c3 = Cube_parametric_typed(1.1,1.2,1.3)
@show volume(c1) == volume(c2) == volume(c3)

volume(c1) == volume(c2) == volume(c3) = true


true

In [44]:
using BenchmarkTools
@btime volume(c1) # not typed
@btime volume(c2) # typed float
@btime volume(c3) # typed parametric

  26.116 ns (1 allocation: 16 bytes)
  6.881 ns (1 allocation: 16 bytes)
  17.550 ns (1 allocation: 16 bytes)


1.7160000000000002

In [47]:
c4 = Cube_parametric_typed{Float64}(1.1,1.2,1.3)
@btime volume(c4) 

  19.667 ns (1 allocation: 16 bytes)


1.7160000000000002

The second and the third function calls are faster! Let's call `@code_warntype` and check type stability

In [48]:
@code_warntype volume(c1)

MethodInstance for volume(::Cube)
  from volume(c::Cube) in Main at In[40]:1
Arguments
  #self#::Core.Const(volume)
  c::Cube
Body::Any
1 ─ %1 = Base.getproperty(c, :length)::Any
│   %2 = Base.getproperty(c, :width)::Any
│   %3 = Base.getproperty(c, :height)::Any
│   %4 = (%1 * %2 * %3)::Any
└──      return %4



In [49]:
@code_warntype volume(c2)

MethodInstance for volume(::Cube_typed)
  from volume(c::Cube_typed) in Main at In[41]:6
Arguments
  #self#::Core.Const(volume)
  c::Cube_typed
Body::Float64
1 ─ %1 = Base.getproperty(c, :length)::Float64
│   %2 = Base.getproperty(c, :width)::Float64
│   %3 = Base.getproperty(c, :height)::Float64
│   %4 = (%1 * %2 * %3)::Float64
└──      return %4



In [50]:
@code_warntype volume(c3)

MethodInstance for volume(::Cube_parametric_typed{Float64})
  from volume(c::Cube_parametric_typed) in Main at In[42]:6
Arguments
  #self#::Core.Const(volume)
  c::Cube_parametric_typed{Float64}
Body::Float64
1 ─ %1 = Base.getproperty(c, :length)::Float64
│   %2 = Base.getproperty(c, :width)::Float64
│   %3 = Base.getproperty(c, :height)::Float64
│   %4 = (%1 * %2 * %3)::Float64
└──      return %4



**Conclusion**: Les types en Julia sont importants donc si vous les connaissez, ajoutez-les, cela peut améliorer les performances.

In [51]:
function zero_or_val(x::Real)
    if x >= 0
        return x
    else
        return 0
    end
end
@code_warntype zero_or_val(0.2)

MethodInstance for zero_or_val(::Float64)
  from zero_or_val(x::Real) in Main at In[51]:1
Arguments
  #self#::Core.Const(zero_or_val)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0



In [52]:
function zero_or_val_stable(x::Real)
    if x >= 0
        y = x
    else
        y = 0
    end
    T = promote_type(typeof(x),Int)
    return T(y)
end
@code_warntype zero_or_val_stable(0.2)

MethodInstance for zero_or_val_stable(::Float64)
  from zero_or_val_stable(x::Real) in Main at In[52]:1
Arguments
  #self#::Core.Const(zero_or_val_stable)
  x::Float64
Locals
  T::Type{Float64}
  y::Union{Float64, Int64}
Body::Float64
1 ─       Core.NewvarNode(:(T))
│         Core.NewvarNode(:(y))
│   %3  = (x >= 0)::Bool
└──       goto #3 if not %3
2 ─       (y = x)
└──       goto #4
3 ─       (y = 0)
4 ┄ %8  = Main.typeof(x)::Core.Const(Float64)
│         (T = Main.promote_type(%8, Main.Int))
│   %10 = (T::Core.Const(Float64))(y)::Float64
└──       return %10



**Conclusion**: `promote_type` peut vous permettre de supprimer une instabilité de type en utilisant la réprésentation la plus haute dans l'abre des types.

Je vous propose le jeu suivant: Soit un vecteur de nombres. Calculons la somme comme suit. 
Pour chaque nombre du vecteur, on lance une pièce de monnaie (`rand()`), si c'est face (`>=0.5`), vous ajoutez `1`. Sinon, vous ajoutez le nombre lui-même.


In [53]:
function flipcoin_then_add(v::Vector{T}) where T <: Real
    s = 0
    for vi in v
        r = rand()
        if r >=0.5
            s += 1
        else
            s += vi
        end
    end
end

function flipcoin_then_add_typed(v::Vector{T}) where T <: Real
    s = zero(T)
    for vi in v
        r = rand()
        if r >=0.5
            s += one(T)
        else
            s += vi
        end
    end
end
myvec = rand(1000)
@show flipcoin_then_add(myvec) == flipcoin_then_add_typed(myvec)

flipcoin_then_add(myvec) == flipcoin_then_add_typed(myvec) = true


true

In [54]:
@btime flipcoin_then_add(rand(1000))
@btime flipcoin_then_add_typed(rand(1000))

  2.434 μs (1 allocation: 7.94 KiB)
  2.401 μs (1 allocation: 7.94 KiB)


**Conclusion**: Think about the variables you are declaring. Do you know their types? If so, specify the type somehow.

### @code_XXX

Nous avons vu durant ce chapitre que regarder le code généré peut nous aider à améliorer les performances. Voici toutes les macros à votre disposition:

In [55]:
# @code_llvm 
# @code_lowered 
# @code_native 
# @code_typed 
# @code_warntype

function flipcoin(randval::Float64)
    if randval<0.5
        return "H"
    else
        return "T"
    end
end

flipcoin (generic function with 1 method)

In [56]:
@code_lowered flipcoin(rand()) # syntax tree

CodeInfo(
1 ─ %1 = randval < 0.5
└──      goto #3 if not %1
2 ─      return "H"
3 ─      return "T"
)

In [57]:
@code_warntype flipcoin(rand()) # try @code_typed

MethodInstance for flipcoin(::Float64)
  from flipcoin(randval::Float64) in Main at In[55]:7
Arguments
  #self#::Core.Const(flipcoin)
  randval::Float64
Body::String
1 ─ %1 = (randval < 0.5)::Bool
└──      goto #3 if not %1
2 ─      return "H"
3 ─      return "T"



In [58]:
@code_llvm flipcoin(rand()) # this and code_warntype are probably the most relevant

;  @ In[55]:7 within `flipcoin`
define nonnull {}* @julia_flipcoin_3582(double %0) #0 {
top:
;  @ In[55]:8 within `flipcoin`
; ┌ @ float.jl:444 within `<`
   %1 = fcmp uge double %0, 5.000000e-01
; └
  %spec.select = select i1 %1, {}* inttoptr (i64 4529490128 to {}*), {}* inttoptr (i64 4529490000 to {}*)
;  @ In[55]:9 within `flipcoin`
  ret {}* %spec.select
}


In [59]:
@code_native flipcoin(rand())

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[55]:7 within `flipcoin`
	movabsq	$4526567312, %rax               ## imm = 0x10DCDEF90
	vmovsd	(%rax), %xmm1                   ## xmm1 = mem[0],zero
; │ @ In[55]:8 within `flipcoin`
; │┌ @ float.jl:444 within `<`
	xorl	%ecx, %ecx
	vucomisd	%xmm0, %xmm1
	setbe	%cl
; │└
	shlq	$7, %rcx
	movabsq	$4529490000, %rax               ## imm = 0x10DFA8850
	orq	%rcx, %rax
; │ @ In[55]:9 within `flipcoin`
	retq
	nopl	(%rax)
; └
